In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("../data/Stations.csv")
data.head()

In [ ]:
s = data["Daytime Routes"].str.split(expand=True).apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = "Route"
data = data.join(s)
data.head()

In [ ]:
data = data[["Line", "Route", "Stop Name", "GTFS Latitude", "GTFS Longitude"]]
data.head()

In [ ]:
groups = data.groupby(["Route", "Line"])
groups.get_group(("N", "Astoria"))

In [ ]:
routes_within_line = dict(dict())
for (route, line), group_data in groups:
    if not route in routes_within_line.keys():
        routes_within_line[route] = {line : group_data}
    else:
        routes_within_line[route][line] = group_data

In [ ]:
for rt, val in routes_within_line.items():
    print(rt)
    print(list(val.keys()), "\n")


In [ ]:
line_orders = {
    '2' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Clark St',
            'Eastern Pky',
            'Nostrand'
            ],
    '3' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Eastern Pky',
            'Clark St'],
    '4' : [ 'Jerome Av',
            'Lexington Av',
            'Eastern Pky' ],
    '5' : [ 'Dyre Av',
            'Lenox - White Plains Rd',
            'Jerome Av',
            'Lexington Av', 
            'Eastern Pky',
            'Nostrand'],
    '6' : [ 'Pelham', 'Lexington Av' ],
    'A' : [ '8th Av - Fulton St',
            'Liberty Av',
            'Rockaway' ],
    'B' : [ 'Concourse',
            '8th Av - Fulton St',
            'Queens Blvd',
            '6th Av - Culver',
            'Broadway - Brighton',
            ], 
    'D' : [ 'Concourse',
            '8th Av - Fulton St', 
            '6th Av - Culver', 
            'Queens Blvd',
            '4th Av', 
            'West End',
            'Sea Beach / West End / Culver / Brighton' ],
    'E' : [ 'Queens - Archer',
            'Queens Blvd',
            '8th Av - Fulton St' ],
    'F' : [ 'Queens Blvd', 
            '63rd St',
            '8th Av - Fulton St',
            '6th Av - Culver',
            'Broadway - Brighton',
            'Sea Beach / West End / Culver / Brighton'],
    'G' : [ 'Crosstown', 
            '8th Av - Fulton St',
            '6th Av - Culver' ],
    'J' : [ 'Queens - Archer', 'Jamaica' ],
    'M' : [ 'Queens Blvd', 
            '6th Av - Culver', 
            'Jamaica', 
            'Myrtle Av' ],
    'N' : [ 'Astoria', 
            'Broadway - Brighton',
            'Manhattan Bridge', 
            '4th Av',
            'Sea Beach', 
           'Sea Beach / West End / Culver / Brighton' ],
    'Q' : [ 'Second Av', 
            '63rd St', 
            'Broadway - Brighton', 
            'Manhattan Bridge', 
            'Sea Beach / West End / Culver / Brighton' ],
    
    'R' : [ 'Queens Blvd',
            'Astoria', 
            'Broadway - Brighton',
            'Broadway',
            '4th Av' ],
    'W' : [ 'Astoria',
            'Broadway - Brighton',
            'Broadway' ],
    'Z' : [ 'Queens - Archer', 'Jamaica' ]
}

In [ ]:
full_routes = dict()

single_line_routes = set(routes_within_line.keys()).difference(set(line_orders.keys()))
for rt in single_line_routes:
    key = list(routes_within_line[rt].keys())[0]
    full_routes[rt] = list(routes_within_line[rt][key]["Stop Name"])


for rt, lines in line_orders.items(): 
    full_routes[rt] = []
    for l in lines: 
        full_routes[rt] += list(routes_within_line[rt][l]["Stop Name"])

In [ ]:
full_routes.keys()

In [ ]:
# borough hall needs to be moved above nevins st (route 4, 5)
old_loc = full_routes['4'].index("Borough Hall")
full_routes['4'].pop(old_loc)
new_loc = full_routes['4'].index("Nevins St")
full_routes['4'].insert(new_loc,'Borough Hall')

old_loc = full_routes['5'].index("Borough Hall")
full_routes['5'].pop(old_loc)
new_loc = full_routes['5'].index("Nevins St")
full_routes['5'].insert(new_loc, "Borough Hall")

# Rockaway 67 - Far rockaway needs to be swapped with 90th-116th (route A)
loc_90th = full_routes['A'].index("Beach 90 St")
for i in range(4):
    val = full_routes['A'][loc_90th]
    full_routes['A'].remove(val)
    full_routes['A'].append(val)

# 145th needs to be put in between 155th and 135th (route B)
# remove parkside av, beverly rd, cortelyou avenu h, av j, av m (route B)
full_routes['B'].remove("145 St")
full_routes['B'].insert(full_routes['B'].index("135 St"), "145 St")
full_routes['B'].remove("W 4 St-Wash Sq")
full_routes['B'].insert(full_routes['B'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
full_routes['B'].remove("Parkside Av")
full_routes['B'].remove("Beverley Rd")
full_routes['B'].remove("Cortelyou Rd")
full_routes['B'].remove("Avenue H")
full_routes['B'].remove("Avenue J")
full_routes['B'].remove("Avenue M")
full_routes['B'].remove("Avenue U")
full_routes['B'].remove("Neck Rd")

# 145th needs to be put between 155th and 125th (route D)
# 7th ave after 59th st 
# wash square after 34th 
full_routes['D'].remove("145 St")
full_routes['D'].insert(full_routes['D'].index("125 St"), "145 St")
full_routes['D'].remove("7 Av")
full_routes['D'][full_routes['D'].index("W 4 St-Wash Sq")] = "7 Av"
full_routes['D'].insert(full_routes['D'].index('Broadway-Lafayette St'), "W 4 St-Wash Sq")

# ROUTE F
# 'Jay St-MetroTech' before bergen
# 'W 4 St-Wash Sq', before 'Broadway-Lafayette St'
full_routes['F'].remove("Jay St-MetroTech")
full_routes['F'].insert(full_routes['F'].index("Bergen St"), "Jay St-MetroTech")
full_routes['F'].remove("W 4 St-Wash Sq")
full_routes['F'].insert(full_routes['F'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")


# ROUTE M 
#  'W 4 St-Wash Sq', after 14th 
# reverse Myrtle - essex and metropolitan - central
full_routes['M'].remove("W 4 St-Wash Sq")
full_routes['M'].insert(full_routes['M'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
reverse_1 = reversed(full_routes['M'][-13:-7])
reverse_2 = reversed(full_routes['M'][-7:])
full_routes['M'] = full_routes['M'][:-13]
full_routes['M'] += reverse_1
full_routes['M'] += reverse_2

# ROUTE N 
# queenseboro after 39th 
full_routes["N"].remove("Queensboro Plaza")
full_routes["N"].insert(full_routes["N"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE Q
# canal after union square 
full_routes["Q"].remove("Canal St")
full_routes["Q"].insert(full_routes['Q'].index("DeKalb Av"), "Canal St")

# ROUTE R
# dekalb after jay street 
full_routes['R'].remove("DeKalb Av")
full_routes['R'].insert(full_routes['R'].index("Atlantic Av-Barclays Ctr"), "DeKalb Av")

# ROUTE W
# queensboro after 39th
full_routes["W"].remove("Queensboro Plaza")
full_routes["W"].insert(full_routes["W"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE S 
# broken into separate shuttles

In [ ]:
stations = data.drop("Line", axis=1)
stations.head()

In [ ]:
edges = pd.DataFrame()
edges.reindex(["Route", "To", "From"])
for key, val in full_routes.items():
    cur_route = pd.DataFrame()
    cur_route["To"] = val
    cur_route["From"] = val
    cur_route["Route"] = key
    cur_route["From"] = cur_route["From"].shift(-1)
    cur_route.dropna(inplace=True)    
    edges = edges.append(cur_route)
    
edges = pd.merge(
            edges, stations, left_on=["Route", "To"], right_on=["Route", "Stop Name"]
        ).drop("Stop Name", axis=1
        ).rename({"GTFS Latitude" : "Latitude_1",
                  "GTFS Longitude" : "Longitude_1"},
                axis=1)

edges = pd.merge(
            edges, stations, left_on=["Route", "From"], right_on=["Route", "Stop Name"]
        ).drop("Stop Name", axis=1
        ).rename({"GTFS Latitude" : "Latitude_2",
                  "GTFS Longitude" : "Longitude_2"},
                axis=1)

edges["Distance"] = np.sqrt(
                        (edges["Latitude_1"]-edges["Latitude_2"])**2 +
                        (edges["Longitude_1"]-edges["Longitude_2"])**2
                    )

edges.head()

In [ ]:
edges[["Route", "To", "From", "Distance"]].to_csv("../data/edges.csv")
stations.to_csv("../data/station_data.csv")